This examples show how to use MiniSom to solve a classification problem. The classification mechanism will be implemented with MiniSom and the evaluation will make use of sklearn.

First, let's load a dataset and scale its columns:

In [33]:
from minisom import MiniSom
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

columns=['area', 'perimeter', 'compactness', 'length_kernel', 'width_kernel',
                   'asymmetry_coefficient', 'length_kernel_groove', 'target']
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt', 
                    names=columns, 
                   sep='\t+', engine='python')
labels = data['target'].values
data = scale(data.values)

Here's naive classification function that classifies a sample in `data` using the label assigned to the associated winning neuron. A label $c$ is associated to a neuron if the majority of samples mapped in that neuron have label $c$. The function will assign the most common label in the dataset in case that a sample is mapped to a neuron for which no class is assigned.

In [34]:
def classify(som, data):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = som.labels_map(X_train, y_train)
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

Now we can 1) split the data in train and test set, 2) train the som, 3) print the classification report that contains all the metrics to evaluate the results of the classification.

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(data, labels, stratify=labels)

som = MiniSom(7, 7, data.shape[1], sigma=3, learning_rate=0.5, 
              neighborhood_function='triangle', random_seed=10)
som.pca_weights_init(X_train)
som.train_random(X_train, 500, verbose=False)

print(classification_report(y_test, classify(som, X_test)))

              precision    recall  f1-score   support

           1       0.94      1.00      0.97        17
           2       1.00      1.00      1.00        18
           3       1.00      0.94      0.97        18

    accuracy                           0.98        53
   macro avg       0.98      0.98      0.98        53
weighted avg       0.98      0.98      0.98        53



../minisom.py:389: ComplexWarning: Casting complex values to real discards the imaginary part
  c2*pc[pc_order[1]]
